In [4]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [5]:
spark = SparkSession.builder \
    .master("local") \
    .appName("ok") \
    .getOrCreate()

In [6]:
from pyspark.sql.types import StructType,StructField, IntegerType
import datetime

now = datetime.datetime.now()

data = [(now.year,now.month,now.day)]

schema = StructType([ \
    StructField("year",IntegerType(),True), \
    StructField("month",IntegerType(),True), \
    StructField("day",IntegerType(),True), \
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2021|    6|  6|
+----+-----+---+



In [7]:
df.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_download_date',
      user='admin',
      password='password'
      ).mode('append').save()